In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, count, countDistinct, col, desc, rank
from pyspark.sql.window import Window

In [ ]:
spark = SparkSession.builder \
    .appName("ETL to ClickHouse") \
    .config("spark.jars", "/home/jovyan/jars/postgresql-42.7.3.jar,/home/jovyan/jars/clickhouse-jdbc-0.4.6.jar") \
    .getOrCreate()

In [ ]:
jdbc_url = "jdbc:postgresql://bigdata_postgres:5432/postgres"
properties = {
    "user": "bd_user",
    "password": "bd_pass",
    "driver": "org.postgresql.Driver"
}

In [ ]:
fact_sales = spark.read.jdbc(url=jdbc_url, table="fact_sales", properties=properties)
dim_product = spark.read.jdbc(url=jdbc_url, table="dim_product", properties=properties)
dim_customer = spark.read.jdbc(url=jdbc_url, table="dim_customer", properties=properties)
dim_time = spark.read.jdbc(url=jdbc_url, table="dim_time", properties=properties)
dim_store = spark.read.jdbc(url=jdbc_url, table="dim_store", properties=properties)
dim_supplier = spark.read.jdbc(url=jdbc_url, table="dim_supplier", properties=properties)

In [ ]:
clickhouse_url = "jdbc:clickhouse://clickhouse:8123/default"

In [1]:
print("\n\t Витрина продаж по продуктам:")
product_report = fact_sales.alias("f") \
    .join(dim_product.alias("p"), "product_id") \
    .groupBy("p.product_name", "p.product_category") \
    .agg(
        sum("f.quantity").alias("total_quantity_sold"),
        sum("f.total_price").alias("total_revenue"),
        avg("p.product_rating").alias("avg_rating"),
        sum("p.product_reviews").alias("total_reviews")
    )
print("Цель: Анализ выручки, количества продаж и популярности продуктов.")
product_report.show(5)


===> Витрина продаж по продуктам:
Цель: Анализ выручки, количества продаж и популярности продуктов.
+------------+----------------+-------------------+--------------------+--------------------+-------------+
|product_name|product_category|total_quantity_sold|       total_revenue|          avg_rating|total_reviews|
+------------+----------------+-------------------+--------------------+--------------------+-------------+
|    Dog Food|             Toy|               6441|298455.0600000000...|3.046153846153846...|       598730|
|    Dog Food|            Food|               5628|265113.7200000000...|2.886407766990291...|       507690|
|    Dog Food|            Cage|               6214|285186.5700000000...|3.098260869565217...|       522080|
|   Bird Cage|             Toy|               6126|278616.2800000000...|3.129729729729729...|       507840|
|   Bird Cage|            Cage|               6172|294655.3600000000...|3.118421052631578...|       552440|
+------------+----------------+----

In [2]:
print("\n\t ТОП-10 самых продаваемых продуктов:")
top_products = product_report.orderBy(desc("total_revenue")).limit(9)
top_products.show()


===> ТОП-10 самых продаваемых продуктов:
+------------+----------------+-------------------+--------------------+--------------------+-------------+
|product_name|product_category|total_quantity_sold|       total_revenue|          avg_rating|total_reviews|
+------------+----------------+-------------------+--------------------+--------------------+-------------+
|     Cat Toy|            Food|               6847|316687.0800000000...|2.928000000000000...|       654350|
|    Dog Food|             Toy|               6441|298455.0600000000...|3.046153846153846...|       598730|
|   Bird Cage|            Cage|               6172|294655.3600000000...|3.118421052631578...|       552440|
|    Dog Food|            Cage|               6214|285186.5700000000...|3.098260869565217...|       522080|
|   Bird Cage|             Toy|               6126|278616.2800000000...|3.129729729729729...|       507840|
|     Cat Toy|            Cage|               5812|265680.0500000000...|2.948113207547169...| 

In [3]:
print("\n\t Общая выручка по категориям продуктов:")
category_revenue = product_report.groupBy("product_category").agg(sum("total_revenue").alias("total_revenue"))
category_revenue.show()


===> Общая выручка по категориям продуктов:
+----------------+--------------------+
|product_category|       total_revenue|
+----------------+--------------------+
|            Cage|845521.9800000000...|
|            Food|843109.6700000000...|
|             Toy|841220.4700000000...|
+----------------+--------------------+



In [4]:
print("\n\t Средний рейтинг и количество отзывов для каждого продукта:")
ratings_and_reviews = product_report.select("product_name", "avg_rating", "total_reviews")
ratings_and_reviews.show()


===> Средний рейтинг и количество отзывов для каждого продукта:
+------------+--------------------+-------------+
|product_name|          avg_rating|total_reviews|
+------------+--------------------+-------------+
|    Dog Food|3.046153846153846...|       598730|
|    Dog Food|2.886407766990291...|       507690|
|    Dog Food|3.098260869565217...|       522080|
|   Bird Cage|3.129729729729729...|       507840|
|   Bird Cage|3.118421052631578...|       552440|
|   Bird Cage|3.182352941176470...|       540150|
|     Cat Toy|2.948113207547169...|       523020|
|     Cat Toy|2.928000000000000...|       654350|
|     Cat Toy|3.057943925233644...|       570810|
+------------+--------------------+-------------+



In [5]:
print("\n\t Витрина продаж по клиентам:")
customer_report = fact_sales.alias("f") \
    .join(dim_customer.alias("c"), "customer_id") \
    .groupBy("c.customer_first_name", "c.customer_last_name", "c.customer_country") \
    .agg(
        sum("f.total_price").alias("total_spent"),
        count("*").alias("total_orders"),
        avg("f.total_price").alias("avg_order_value")
    )


===> Витрина продаж по клиентам:


In [6]:
print("Цель: Анализ покупательского поведения и сегментация клиентов.")
customer_report.show(5)

Цель: Анализ покупательского поведения и сегментация клиентов.
+-------------------+------------------+----------------+--------------------+------------+--------------------+
|customer_first_name|customer_last_name|customer_country|         total_spent|total_orders|     avg_order_value|
+-------------------+------------------+----------------+--------------------+------------+--------------------+
|           Charlene|           Topping|            Mali|2886.370000000000...|          10|288.6370000000000...|
|               Trey|          Conigsby|          Poland|2013.840000000000...|          10|201.3840000000000...|
|              Roxie|           Marqyes|            Iran|1747.740000000000...|          10|174.7740000000000...|
|               Lacy|          Kilfeder|          Poland|2005.730000000000...|          10|200.5730000000000...|
|          Clementia|             Crate|          Poland|2189.790000000000...|          10|218.9790000000000...|
+-------------------+------------

In [7]:
print("\n\t ТОП-10 клиентов с наибольшей общей суммой покупок:")
top_customers = customer_report.orderBy(desc("total_spent")).limit(10)
top_customers.show()


===> ТОП-10 клиентов с наибольшей общей суммой покупок:
+-------------------+------------------+----------------+--------------------+------------+--------------------+
|customer_first_name|customer_last_name|customer_country|         total_spent|total_orders|     avg_order_value|
+-------------------+------------------+----------------+--------------------+------------+--------------------+
|               Mile|              Tuer|          Jordan|4005.980000000000...|          10|400.5980000000000...|
|             Lewiss|           Pinshon|           China|3784.440000000000...|          10|378.4440000000000...|
|             Alexis|           Quinton|           China|3751.090000000000...|          10|375.1090000000000...|
|             Benita|           Godding|         Nigeria|3682.520000000000...|          10|368.2520000000000...|
|             Elvira|         Faircliff|       Australia|3645.940000000000...|          10|364.5940000000000...|
|          Shandeigh|         Thomassin

In [8]:
print("\n\t Распределение клиентов по странам:")
country_distribution = customer_report.groupBy("customer_country").agg(count("*").alias("clients_count"))
country_distribution.show()


===> Распределение клиентов по странам:
+----------------+-------------+
|customer_country|clients_count|
+----------------+-------------+
|            Chad|            1|
|          Russia|           47|
|           Yemen|            2|
|          Sweden|           28|
|     Philippines|           46|
|        Malaysia|            5|
|            Iraq|            1|
|         Germany|            2|
|         Comoros|            1|
|        Cambodia|            1|
|     Afghanistan|            1|
|          Jordan|            3|
|     Ivory Coast|            3|
|           Sudan|            2|
|          France|           35|
|          Greece|           14|
|          Kosovo|            1|
|       Sri Lanka|            2|
|          Taiwan|            1|
|       Argentina|           10|
+----------------+-------------+
only showing top 20 rows



In [9]:
print("\n\t Средний чек для каждого клиента:")
avg_check_per_customer = customer_report.select("customer_first_name", "customer_last_name", "avg_order_value")
avg_check_per_customer.show()


===> Средний чек для каждого клиента:
+-------------------+------------------+--------------------+
|customer_first_name|customer_last_name|     avg_order_value|
+-------------------+------------------+--------------------+
|           Charlene|           Topping|288.6370000000000...|
|               Trey|          Conigsby|201.3840000000000...|
|              Roxie|           Marqyes|174.7740000000000...|
|               Lacy|          Kilfeder|200.5730000000000...|
|          Clementia|             Crate|218.9790000000000...|
|            Tristam|            Tandey|273.2890000000000...|
|            Abelard|            Guyers|178.5970000000000...|
|              Alfie|          Nannetti|256.6570000000000...|
|          Rosalinde|        O'Hogertie|302.5140000000000...|
|               Lyon|        Beverstock|225.2390000000000...|
|               Dall|             Frear|272.1510000000000...|
|              Rahel|          Thonason|249.1000000000000...|
|             Dickie|          

In [10]:
print("\n\t Витрина продаж по времени:")
time_report = fact_sales.alias("f") \
    .join(dim_time.alias("t"), "time_id") \
    .groupBy("t.year", "t.month") \
    .agg(
        sum("f.total_price").alias("monthly_revenue"),
        sum("f.quantity").alias("total_items_sold"),
        avg("f.total_price").alias("avg_order_size")
    ).orderBy("t.year", "t.month")
print("Цель: Анализ сезонности и трендов продаж.")
time_report.show(5)


===> Витрина продаж по времени:
Цель: Анализ сезонности и трендов продаж.
+----+-----+--------------------+----------------+--------------------+
|year|month|     monthly_revenue|total_items_sold|      avg_order_size|
+----+-----+--------------------+----------------+--------------------+
|2021|    1|224158.5400000000...|            4856|256.4743020594965...|
|2021|    2|192348.3100000000...|            4070|260.2818809201623...|
|2021|    3|207282.2000000000...|            4561|245.8863582443653...|
|2021|    4|206592.8200000000...|            4564|246.8253524492234...|
|2021|    5|211764.8600000000...|            4451|255.7546618357487...|
+----+-----+--------------------+----------------+--------------------+
only showing top 5 rows



In [11]:
print("\n\t Месячные и годовые тренды продаж:")
yearly_trends = time_report.groupBy("year").agg(sum("monthly_revenue").alias("annual_revenue"))
yearly_trends.show()


===> Месячные и годовые тренды продаж:
+----+--------------------+
|year|      annual_revenue|
+----+--------------------+
|2021|2529852.120000000...|
+----+--------------------+



In [12]:
print("\n\t Средний размер заказа по месяцам:")
avg_order_size_per_month = time_report.select("year", "month", "avg_order_size")
avg_order_size_per_month.show()


===> Средний размер заказа по месяцам:
+----+-----+--------------------+
|year|month|      avg_order_size|
+----+-----+--------------------+
|2021|    1|256.4743020594965...|
|2021|    2|260.2818809201623...|
|2021|    3|245.8863582443653...|
|2021|    4|246.8253524492234...|
|2021|    5|255.7546618357487...|
|2021|    6|261.6092457420924...|
|2021|    7|256.9889393939393...|
|2021|    8|246.6842586399108...|
|2021|    9|251.0410369487485...|
|2021|   10|256.4386995515695...|
|2021|   11|249.8810112359550...|
|2021|   12|248.5309870129870...|
+----+-----+--------------------+



In [13]:
print("\n\t Витрина продаж по магазинам:")
store_report = fact_sales.alias("f") \
    .join(dim_store.alias("s"), "store_id") \
    .groupBy("s.store_name", "s.store_city", "s.store_country") \
    .agg(
        sum("f.total_price").alias("total_revenue"),
        count("*").alias("orders_count"),
        avg("f.total_price").alias("avg_order_value")
    )
print("Цель: Анализ эффективности магазинов.")
store_report.show(5)


===> Витрина продаж по магазинам:
Цель: Анализ эффективности магазинов.
+------------+-----------------+-------------+--------------------+------------+--------------------+
|  store_name|       store_city|store_country|       total_revenue|orders_count|     avg_order_value|
+------------+-----------------+-------------+--------------------+------------+--------------------+
|Shuffledrive|Banjar Taro Kelod|        China|30.41000000000000...|           1|30.41000000000000...|
|    Centidel|     Donggaohuang|       France|294.5300000000000...|           1|294.5300000000000...|
|       Eabox|         Ureshino|       Angola|428.0000000000000...|           1|428.0000000000000...|
|    Livepath|          Haljala|       Greece|288.5100000000000...|           1|288.5100000000000...|
|       Lazzy|          Masākin|     Pakistan|138.3900000000000...|           1|138.3900000000000...|
+------------+-----------------+-------------+--------------------+------------+--------------------+
only show

In [14]:
print("\n\t ТОП-5 магазинов с наибольшей выручкой:")
top_stores = store_report.orderBy(desc("total_revenue")).limit(5)
top_stores.show()


===> ТОП-5 магазинов с наибольшей выручкой:
+------------+-------------------+-------------+--------------------+------------+--------------------+
|  store_name|         store_city|store_country|       total_revenue|orders_count|     avg_order_value|
+------------+-------------------+-------------+--------------------+------------+--------------------+
|    Feedspan|             Monywa|      Ukraine|1317.200000000000...|           3|439.0666666666666...|
|      Demizz|              Ranot|       Mexico|1212.270000000000...|           4|303.0675000000000...|
|       Quatz|            Kasungu|    Indonesia|1197.640000000000...|           5|239.5280000000000...|
|     Rhycero|Saint-Jean-de-Braye|       Poland|1069.750000000000...|           3|356.5833333333333...|
|Twitterworks|             Iecava|    Argentina|1005.190000000000...|           3|335.0633333333333...|
+------------+-------------------+-------------+--------------------+------------+--------------------+



In [15]:
print("\n\t Распределение продаж по городам и странам:")
store_distribution = store_report.groupBy("store_city", "store_country").agg(sum("total_revenue").alias("total_revenue"))
store_distribution.show()


===> Распределение продаж по городам и странам:
+-------------------+-------------+--------------------+
|         store_city|store_country|       total_revenue|
+-------------------+-------------+--------------------+
|        Al Jubayhah|  Philippines|266.2700000000000...|
|            Wangren|    Indonesia|481.2800000000000...|
|       Kohtla-Järve|        China|409.3000000000000...|
|           Yanjiang|         Peru|177.9700000000000...|
|            Baidian|      Finland|146.4900000000000...|
|            Golacir|     Thailand|390.4600000000000...|
|               Mary|        Spain|366.8500000000000...|
|              Baras|     Thailand|122.0500000000000...|
|             Dongpu|        China|493.1700000000000...|
|         Champerico|     Portugal|487.9000000000000...|
|           Nanggela|     Thailand|322.3300000000000...|
|             Linshi|United States|56.76000000000000...|
|            Kherson|       Sweden|495.9400000000000...|
|            Xundian|     Colombia|333.

In [16]:
print("\n\t Средний чек для каждого магазина:")
avg_check_per_store = store_report.select("store_name", "avg_order_value")
avg_check_per_store.show()


===> Средний чек для каждого магазина:
+-------------+--------------------+
|   store_name|     avg_order_value|
+-------------+--------------------+
| Shuffledrive|30.41000000000000...|
|     Centidel|294.5300000000000...|
|        Eabox|428.0000000000000...|
|     Livepath|288.5100000000000...|
|        Lazzy|138.3900000000000...|
|        Quinu|56.21000000000000...|
|     Flashdog|331.7100000000000...|
|         Yodo|340.0800000000000...|
|        Quatz|129.0500000000000...|
|        Janyx|353.7600000000000...|
|    Topicblab|366.1400000000000...|
|        Kimia|69.25000000000000...|
|       Wikivu|198.6100000000000...|
|      Teklist|159.1000000000000...|
|       Meedoo|425.1800000000000...|
|        Aimbo|296.9000000000000...|
|        Voomm|186.9700000000000...|
|        Janyx|58.69000000000000...|
|       Wikivu|365.6000000000000...|
|Thoughtsphere|84.54000000000000...|
+-------------+--------------------+
only showing top 20 rows



In [17]:
print("\n\t Витрина продаж по поставщикам:")
supplier_report = fact_sales.alias("f") \
    .join(dim_supplier.alias("sup"), "supplier_id") \
    .groupBy("sup.supplier_name", "sup.supplier_country") \
    .agg(
        sum("f.total_price").alias("total_revenue"),
        avg("f.total_price").alias("avg_product_price")
    )
print("Цель: Анализ эффективности поставщиков.")
supplier_report.show(5)


===> Витрина продаж по поставщикам:
Цель: Анализ эффективности поставщиков.
+-------------+----------------+--------------------+--------------------+
|supplier_name|supplier_country|       total_revenue|   avg_product_price|
+-------------+----------------+--------------------+--------------------+
|   Zoomlounge|           China|220.9000000000000...|220.9000000000000...|
|        InnoZ|        Colombia|123.5200000000000...|123.5200000000000...|
|     Realfire|           Japan|1043.170000000000...|347.7233333333333...|
|     Topdrive|          Sweden|2334.640000000000...|291.8300000000000...|
|        Rooxo|        Pakistan|1419.670000000000...|473.2233333333333...|
+-------------+----------------+--------------------+--------------------+
only showing top 5 rows



In [18]:
print("\n\t ТОП-5 поставщиков с наибольшей выручкой:")
top_suppliers = supplier_report.orderBy(desc("total_revenue")).limit(5)
top_suppliers.show()


===> ТОП-5 поставщиков с наибольшей выручкой:
+-------------+----------------+--------------------+--------------------+
|supplier_name|supplier_country|       total_revenue|   avg_product_price|
+-------------+----------------+--------------------+--------------------+
|        Twimm|    Burkina Faso|4955.940000000000...|353.9957142857142...|
|       Trudeo|         Morocco|3944.050000000000...|262.9366666666666...|
|  Brainlounge|          Panama|3684.540000000000...|263.1814285714285...|
|      Myworks|         Nigeria|3642.770000000000...|331.1609090909090...|
|        Vidoo|      Bangladesh|3336.920000000000...|303.3563636363636...|
+-------------+----------------+--------------------+--------------------+



In [19]:
print("\n\t Средняя цена товаров от каждого поставщика:")
avg_price_per_supplier = supplier_report.select("supplier_name", "avg_product_price")
avg_price_per_supplier.show()


===> Средняя цена товаров от каждого поставщика:
+-------------+--------------------+
|supplier_name|   avg_product_price|
+-------------+--------------------+
|   Zoomlounge|220.9000000000000...|
|        InnoZ|123.5200000000000...|
|     Realfire|347.7233333333333...|
|     Topdrive|291.8300000000000...|
|        Rooxo|473.2233333333333...|
|         Lajo|315.0100000000000...|
|        Minyx|367.7800000000000...|
|Twitternation|129.8400000000000...|
|      Youspan|309.9733333333333...|
|      Trilith|152.5500000000000...|
|     Gigazoom|127.0600000000000...|
|        Jazzy|123.0200000000000...|
|  Twitterlist|78.78000000000000...|
|         Kazu|488.9800000000000...|
|     Edgeclub|285.5600000000000...|
|    Zoonoodle|451.3100000000000...|
|       Zoozzy|38.54000000000000...|
|      Teklist|270.8200000000000...|
|       Skivee|478.6000000000000...|
|        Quaxo|367.4475000000000...|
+-------------+--------------------+
only showing top 20 rows



In [20]:
print("\n\t Распределение продаж по странам поставщиков:")
supplier_distribution = supplier_report.groupBy("supplier_country").agg(sum("total_revenue").alias("total_revenue"))
supplier_distribution.show()


===> Распределение продаж по странам поставщиков:
+--------------------+--------------------+
|    supplier_country|       total_revenue|
+--------------------+--------------------+
|                Chad|2078.750000000000...|
|              Russia|127983.9000000000...|
|            Paraguay|4269.170000000000...|
|               Yemen|14436.57000000000...|
|             Senegal|3749.030000000000...|
|              Sweden|72716.27000000000...|
|             Tokelau|461.8200000000000...|
|            Kiribati|63.29000000000000...|
|French Southern T...|217.3000000000000...|
|              Guyana|937.5700000000000...|
|         Philippines|117536.9700000000...|
|             Eritrea|694.4000000000000...|
|               Tonga|1206.200000000000...|
|            Djibouti|440.6100000000000...|
|            Malaysia|23199.83000000000...|
|                Fiji|231.8700000000000...|
|              Turkey|418.4100000000000...|
|              Malawi|4294.810000000000...|
|                Iraq|435

In [21]:
print("\n\t Витрина качества продукции:")
quality_report = dim_product.groupBy("product_category") \
    .agg(
        avg("product_rating").alias("avg_category_rating"),
        sum("product_reviews").alias("total_category_reviews")
    )
print("Цель: Анализ отзывов и рейтингов товаров.")
quality_report.show(5)


===> Витрина качества продукции:
Цель: Анализ отзывов и рейтингов товаров.
+----------------+--------------------+----------------------+
|product_category| avg_category_rating|total_category_reviews|
+----------------+--------------------+----------------------+
|            Cage|3.057611940298507...|                159754|
|            Food|2.993636363636363...|                170219|
|             Toy|3.077611940298507...|                167738|
+----------------+--------------------+----------------------+



In [22]:
print("\n\t Продукты с наивысшим и наименьшим рейтингом:")
highest_lowest_ratings = dim_product.sort(desc("product_rating")).limit(5)
highest_lowest_ratings.show()


===> Продукты с наивысшим и наименьшим рейтингом:
+----------+------------+----------------+--------------------+--------------------+-------------+------------+-------------+----------------+--------------------+--------------------+---------------+--------------------+-------------------+
|product_id|product_name|product_category|       product_price|      product_weight|product_color|product_size|product_brand|product_material| product_description|      product_rating|product_reviews|product_release_date|product_expiry_date|
+----------+------------+----------------+--------------------+--------------------+-------------+------------+-------------+----------------+--------------------+--------------------+---------------+--------------------+-------------------+
|       182|    Dog Food|             Toy|57.85000000000000...|18.40000000000000...|       Purple|       Small|     Tanoodle|           Brass|Praesent blandit....|5.000000000000000000|             24|          2019-04-08|  

In [23]:
print("\n\t Корреляция между рейтингом и объемом продаж:")
correlation = dim_product.corr("product_rating", "product_reviews")
print(f"Корреляция рейтинга и количества отзывов: {correlation:.3f}")


===> Корреляция между рейтингом и объемом продаж:
Корреляция рейтинга и количества отзывов: 0.012


In [24]:
print("\n\t Продукты с наибольшим количеством отзывов:")
most_reviewed_products = dim_product.orderBy(desc("product_reviews")).limit(5)
most_reviewed_products.show()


===> Продукты с наибольшим количеством отзывов:
+----------+------------+----------------+--------------------+--------------------+-------------+------------+-------------+----------------+--------------------+--------------------+---------------+--------------------+-------------------+
|product_id|product_name|product_category|       product_price|      product_weight|product_color|product_size|product_brand|product_material| product_description|      product_rating|product_reviews|product_release_date|product_expiry_date|
+----------+------------+----------------+--------------------+--------------------+-------------+------------+-------------+----------------+--------------------+--------------------+---------------+--------------------+-------------------+
|       732|   Bird Cage|            Food|55.84000000000000...|25.40000000000000...|       Orange|      Medium|       Rhyloo|         Granite|Nulla ut erat id ...|4.000000000000000000|           1000|          2014-10-18|    

In [ ]:
reports = {
    "product_sales_mart": product_report,
    "top_products": top_products,
    "time_sales_mart": time_report,
    "customer_sales_mart": customer_report,
    "store_sales_mart": store_report,
    "supplier_sales_mart": supplier_report,
    "product_quality_mart": quality_report
}

In [ ]:
for table_name, df in reports.items():
    engine = "MergeTree() ORDER BY tuple()"
    
    if table_name in ["product_quality_mart"]:
        engine = "TinyLog()"
    
    df.write \
        .format("jdbc") \
        .option("url", clickhouse_url) \
        .option("dbtable", table_name) \
        .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
        .option("createTableOptions", f"ENGINE = {engine}") \
        .mode("overwrite") \
        .save()